In [19]:
import numpy as np

In [20]:
def sigmoid(x):
  # Sigmoid activation function: f(x) = 1 / (1 + e^(-x))
  return 1 / (1 + np.exp(-x))

In [21]:
def deriv_sigmoid(x):
  # Derivative of sigmoid: f'(x) = f(x) * (1 - f(x))
  fx = sigmoid(x)
  return fx * (1 - fx)


In [22]:
def mse_loss(y_true, y_pred):
  # y_true and y_pred are numpy arrays of the same length.
  return ((y_true - y_pred) ** 2).mean()

In [23]:
class OurNeuralNetwork:
  '''
  A neural network with:
    - 2 inputs
    - a hidden layer with 2 neurons (h1, h2)
    - an output layer with 1 neuron (o1)
  *** DISCLAIMER ***:
  The code below is intended to be simple and educational, NOT optimal.
  Real neural net code looks nothing like this. DO NOT use this code.
  Instead, read/run it to understand how this specific network works.
  '''
  def __init__(self):
    # Weights
    self.w1 = np.random.normal()
    self.w2 = np.random.normal()
    self.w3 = np.random.normal()
    self.w4 = np.random.normal()
    self.w5 = np.random.normal()
    self.w6 = np.random.normal()

    # Biases
    self.b1 = np.random.normal()
    self.b2 = np.random.normal()
    self.b3 = np.random.normal()

  def feedforward(self, x):
    # x is a numpy array with 2 elements.
    h1 = sigmoid(self.w1 * x[0] + self.w2 * x[1] + self.b1)
    h2 = sigmoid(self.w3 * x[0] + self.w4 * x[1] + self.b2)
    o1 = sigmoid(self.w5 * h1 + self.w6 * h2 + self.b3)
    return o1

  def train(self, data, all_y_trues):
    
    learn_rate = 0.001
    epochs = 10000 # number of times to loop through the entire dataset
    losses=[]

    for epoch in range(epochs):
      for x, y_true in zip(data, all_y_trues):
        # --- Do a feedforward (we'll need these values later)
        sum_h1 = self.w1 * x[0] + self.w2 * x[1] + self.b1
        h1 = sigmoid(sum_h1)

        sum_h2 = self.w3 * x[0] + self.w4 * x[1] + self.b2
        h2 = sigmoid(sum_h2)

        sum_o1 = self.w5 * h1 + self.w6 * h2 + self.b3
        o1 = sigmoid(sum_o1)
        y_pred = o1

        # --- Calculate partial derivatives.
        # --- Naming: d_L_d_w1 represents "partial L / partial w1"
        d_L_d_ypred = -2 * (y_true - y_pred)

        # Neuron o1
        d_ypred_d_w5 = h1 * deriv_sigmoid(sum_o1)
        d_ypred_d_w6 = h2 * deriv_sigmoid(sum_o1)
        d_ypred_d_b3 = deriv_sigmoid(sum_o1)

        d_ypred_d_h1 = self.w5 * deriv_sigmoid(sum_o1)
        d_ypred_d_h2 = self.w6 * deriv_sigmoid(sum_o1)

        # Neuron h1
        d_h1_d_w1 = x[0] * deriv_sigmoid(sum_h1)
        d_h1_d_w2 = x[1] * deriv_sigmoid(sum_h1)
        d_h1_d_b1 = deriv_sigmoid(sum_h1)

        # Neuron h2
        d_h2_d_w3 = x[0] * deriv_sigmoid(sum_h2)
        d_h2_d_w4 = x[1] * deriv_sigmoid(sum_h2)
        d_h2_d_b2 = deriv_sigmoid(sum_h2)

        # --- Update weights and biases
        # Neuron h1
        self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
        self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w2
        self.b1 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_b1

        # Neuron h2
        self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w3
        self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
        self.b2 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_b2

        # Neuron o1
        self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_w5
        self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w6
        self.b3 -= learn_rate * d_L_d_ypred * d_ypred_d_b3

      # --- Calculate total loss at the end of each epoch
      if epoch % 200 == 0:
        y_preds = np.apply_along_axis(self.feedforward, 1, data)
        loss = mse_loss(all_y_trues, y_preds)
        losses.append(loss)
        print("Epoch %d loss: %.3f" % (epoch, loss))
        
    return losses

In [24]:
# data = np.array([
#   [-3, -12],  # Rachel
#   [-8, 3],   # Monica
#   [14, 1],   # Joey
#   [3, 9], # Rose
# ])
# all_y_trues = np.array([
#   0, # Rachel
#   0, # Monica
#   1, # Joey
#   1, # Rose
# ])



with open("gender.csv") as f: # this should be done in pandas but i wanted to stay as vanilla as possible
    contents = f.read().split("\n")
    original_data = [x.split(",") for x in contents]
    for person in original_data:
        for index, nums in enumerate(person):
          if index == 1:
            if person[index - 1] == 1:
              person[index] = int(nums) - 140
            else:
              person[index] = int(nums) - 170
          elif index == 2:
            if person[index - 1] == 1:
              person[index] = int(nums) - 80
            else:
              person[index] = int(nums) - 90
          else:
            person[index] = int(nums)


    all_y_trues = []
    data = []
    for person in original_data:
        data.append(person[1:])
        all_y_trues.append(person[0])



#
data = np.array(data)


all_y_trues = np.array(all_y_trues)

In [25]:
network = OurNeuralNetwork()

In [26]:
losses=network.train(data, all_y_trues)

Epoch 0 loss: 0.265
Epoch 200 loss: 0.180
Epoch 400 loss: 0.175
Epoch 600 loss: 0.173
Epoch 800 loss: 0.172
Epoch 1000 loss: 0.172
Epoch 1200 loss: 0.171
Epoch 1400 loss: 0.171
Epoch 1600 loss: 0.171
Epoch 1800 loss: 0.171
Epoch 2000 loss: 0.171
Epoch 2200 loss: 0.171
Epoch 2400 loss: 0.171
Epoch 2600 loss: 0.171
Epoch 2800 loss: 0.171
Epoch 3000 loss: 0.171
Epoch 3200 loss: 0.170
Epoch 3400 loss: 0.170
Epoch 3600 loss: 0.170
Epoch 3800 loss: 0.170
Epoch 4000 loss: 0.170
Epoch 4200 loss: 0.170
Epoch 4400 loss: 0.170
Epoch 4600 loss: 0.170
Epoch 4800 loss: 0.170
Epoch 5000 loss: 0.170
Epoch 5200 loss: 0.170
Epoch 5400 loss: 0.170
Epoch 5600 loss: 0.170
Epoch 5800 loss: 0.170
Epoch 6000 loss: 0.170
Epoch 6200 loss: 0.170
Epoch 6400 loss: 0.170
Epoch 6600 loss: 0.170
Epoch 6800 loss: 0.170
Epoch 7000 loss: 0.170
Epoch 7200 loss: 0.170
Epoch 7400 loss: 0.170
Epoch 7600 loss: 0.170
Epoch 7800 loss: 0.170
Epoch 8000 loss: 0.170
Epoch 8200 loss: 0.170
Epoch 8400 loss: 0.170
Epoch 8600 loss: 0